In [1]:
# Import necessary libraries

In [2]:
pip install opencv-python -q

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install matplotlib -q

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install keras -q

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install numpy -q

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install tensorflow -q

Note: you may need to restart the kernel to use updated packages.


In [7]:
%%writefile emotion_detection.py

import cv2
import numpy as np
from keras.models import load_model
from keras.preprocessing.image import img_to_array

 # This function should implement emotion detection logic.
    # Input 'frame' is an image frame to analyze.
    # The result 'emotion_result' should represent the detected emotion.
def detect_emotion(frame):

    return emotion_result

Overwriting emotion_detection.py


In [8]:
from keras.models import load_model
from time import sleep
from keras.preprocessing.image import img_to_array
from keras.preprocessing import image


In [18]:
import cv2
import numpy as np
from keras.models import load_model
from keras.preprocessing.image import img_to_array
import os
import matplotlib.pyplot as plt

# Load the face cascade classifier and emotion detection model
face_cascade = cv2.CascadeClassifier(r'C:\Users\simon\OneDrive\Skrivbord\haarcascade_frontalface_default.xml')
classifier = load_model(r'C:\Users\simon\OneDrive\Skrivbord\model.h5')

# Define emotion labels
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']

# Skapa en mapp för att spara bilder och videor om den inte redan finns
output_folder = (r'C:\Users\simon\Printed images and videos')
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Create a VideoCapture object to access the camera
cap = cv2.VideoCapture(0)

# Create a VideoWriter to record video when a specific emotion is detected
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(os.path.join(output_folder, 'output.avi'), fourcc, 20.0, (640, 480))

# Initialize counters for images and videos for each emotion
emotion_image_counts = {label: 0 for label in emotion_labels}
emotion_video_counts = {label: 0 for label in emotion_labels}

while True:
    # Read a frame from the camera
    ret, frame = cap.read()

    if not ret:
        break

    labels = []
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray)

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 255), 2)
        roi_gray = gray[y:y + h, x:x + w]
        roi_gray = cv2.resize(roi_gray, (48, 48), interpolation=cv2.INTER_AREA)

        if np.sum([roi_gray]) != 0:
            roi = roi_gray.astype('float') / 255.0
            roi = img_to_array(roi)
            roi = np.expand_dims(roi, axis=0)

            prediction = classifier.predict(roi)[0]
            label = emotion_labels[prediction.argmax()]
            label_position = (x, y)
            cv2.putText(frame, label, label_position, cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            # Capture a photo when an emotion is detected
            if emotion_image_counts[label] < 3:
                emotion_image_counts[label] += 1
                image_filename = os.path.join(output_folder, f'{label.lower()}_face_{emotion_image_counts[label]}.png')
                cv2.imwrite(image_filename, frame)

            # Record video when an emotion is detected
            if emotion_video_counts[label] < 3:
                emotion_video_counts[label] += 1
                out.write(frame)

        else:
            cv2.putText(frame, 'No Faces', (30, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Display the frame using cv2.imshow() in a separate window
    cv2.imshow('Emotion Detector', frame)

    # Exit the loop when the 'q' key is pressed or all emotion limits are reached
    if cv2.waitKey(1) & 0xFF == ord('q') or all(count >= 5 for count in emotion_image_counts.values()):
        break

# Release and clean up resources
cap.release()
out.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 25ms/step
